As colunas que serão ultizados nas análises

In [1]:
colunas = {
    'Estrato': 'Estrato',
    'V1022': 'Situação do domicílio',
    'A001A': 'Condição no domicílio',
    'A002': 'Idade do morador',
    'A003': 'Sexo',
    'A004': 'Cor ou raça',
    'A005': 'Escolaridade',
    'A006': 'Frequenta escola',
    'A006A': 'A escola / escola ou faculdade que frequenta é pública ou privada?',
    'A006B': 'Você está tendo aulas presenciais?',
    'B0011': 'Na semana passada teve febre?',
    'B0012': 'Na semana passada teve tosse?',
    'B0013': 'Na semana passada teve dor de garganta?',
    'B0014': 'Na semana passada teve dificuldade para respirar?',
    'B0015': 'Na semana passada teve dor de cabeça?',
    'B0016': 'Na semana passada teve dor no peito?',
    'B0017': 'Na semana passada teve náusea?',
    'B0018': 'Na semana passada teve nariz entupido ou escorrendo?',
    'B0019': 'Na semana passada teve fadiga?',
    'B00110': 'Na semana passada teve dor nos olhos?',
    'B00111': 'Na semana passada teve perda de cheiro ou sabor?',
    'B00112': 'Na semana passada teve dor muscular?',
    'B00113': 'Na semana passada teve diarreia?',
    'B008': 'O(A) Sr(a) fez algum teste para saber se estava infectado(a) pelo coronavírus?',
    'B009A': 'Fez o exame coletado com cotonete na boca e/ou nariz (SWAB)?',
    'B009B': 'Qual o resultado?',
    'B009C': 'Fez o exame de coleta de sangue através de furo no dedo?',
    'B009D': 'Qual o resultado?',
    'B009E': 'Fez o exame de coleta de sangue através da veia do braço?',
    'B009F': 'Qual o resultado?',
    'C007': 'No trabalho (único ou principal) que tinha nessa semana, era:',
    'C007A': 'Esse trabalho era na área:',
    'C007B': 'Tem carteira de trabalho assinada ou é funcionário público estatutário?',
    'C007C': 'Que tipo de trabalho, cargo ou função você realiza no seu trabalho (único ou principal)?',
    'C007D': 'Qual é a principal atividade do local ou empresa em que você trabalha?',
    'C01011': 'Número da faixa do rendimento/retirada em dinheiro',
    'C01012': 'Valor em dinheiro',
    'F001': 'Este domicílio é:'
}

Mapeamento dos dados

In [2]:
mapeamento_situacao_domicilio = {
    1: 'Urbana',
	2: 'Rural'
}

mapeamento_condicao_domicilio = {
    1: 'Pessoa responsável pelo domicílio',
    2: 'Cônjuge ou companheiro(a) de sexo diferente',
    3: 'Cônjuge ou companheiro(a) do mesmo sexo',
    4: 'Filho(a) do responsável e do cônjuge',
    5: 'Filho(a) somente do responsável',
    6: 'Filho(a) somente do cônjuge',
    7: 'Genro ou nora',
    8: 'Pai, mãe, padrasto ou madrasta',
    9: 'Sogro(a)',
    10: 'Neto(a)',
    11: 'Bisneto(a)',
    12: 'Irmão ou irmã',
    13: 'Avô ou avó',
    14: 'Outro parente',
    15: 'Agregado(a) - Não parente que não compartilha despesas',
    16: 'Convivente - Não parente que compartilha despesas',
    17: 'Pensionista',
    18: 'Empregado(a) doméstico(a)',
    19: 'Parente do(a) empregado(a) doméstico(a)'
}

mapeamento_sexo = {
    1: 'Homem',
	2: 'Mulher'
}

mapeamento_cor_raca = {
    1: 'Branca',
    2: 'Preta',
    3: 'Amarela',
    4: 'Parda',
    5: 'Indígena',
    9: 'Ignorado'
}

mapeamento_escolaridade = {
    1: 'Sem instrução',
    2: 'Fundamental incompleto',
    3: 'Fundamental completa',
    4: 'Médio incompleto',
    5: 'Médio completo',
    6: 'Superior incompleto',
    7: 'Superior completo',
    8: 'Pós-graduação, mestrado ou doutorado'
}

mapeamento_frequenta_escola = {
    1: 'Sim',
    2: 'Não'
}

mapeamento_escola_publica_privada = {
    1: 'Pública',
    2: 'Privada'
}

mapeamento_aulas_presenciais = {
    1: 'Sim, normalmente',
    2: 'Sim, mas apenas parcialmente',
    3: 'Não, e meu normalmente é presencial/semipresencial',
    4: 'Não, meu curso é online'
}

mapeamento_sintomas = {
    1: 'Sim',
    2: 'Não',
    3: 'Não sabe',
    9: 'Ignorado'
}

mapeamento_teste_covid = {
    1: 'Sim',
    2: 'Não',
    9: 'Ignorado'
}

mapeamento_resultado_teste = {
    1: 'Positivo',
    2: 'Negativo',
    3: 'Inconclusivo',
    4: 'Ainda não recebeu o resultado',
    9: 'Ignorado'
}

mapeamento_tipo_trabalho = {
    1: 'Trabalhador doméstico (empregado doméstico, cuidados, babá)',
    2: 'Militar do exército, marinha ou aeronáutica',
    3: 'Policial militar ou bombeiro militar',
    4: 'Empregado do setor privado',
    5: 'Empregado do setor público (inclusive empresas de economia mista)',
    6: 'Empregador',
    7: 'Conta própria',
    8: 'Trabalhador familiar não remunerado em ajuda a membro do domicílio ou parente',
    9: 'Estava fora do mercado de trabalho (fazia apenas afazeres domésticos, cuidados de pessoas ou produção para próprio consumo)'
}

mapeamento_area_trabalho = {
    1: 'Federal',
    2: 'Estadual',
    3: 'Municipal'
}

mapeamento_carteira_trabalho = {
    1: 'Sim, tem carteira de trabalho assinada',
    2: 'Sim, é servidor público estatutário',
    3: 'Não'
}

mapeamento_tipo_cargo = {
    1: 'Empregado doméstico, diarista, cozinheiro (em domicílios particulares)',
    2: 'Faxineiro, auxiliar de limpeza etc. (em empresa pública ou privada)',
    3: 'Auxiliar de escritório, escriturário',
    4: 'Secretária, recepcionista',
    5: 'Operador de Telemarketing',
    6: 'Comerciante (dono do bar, da loja etc.)',
    7: 'Balconista, vendedor de loja',
    8: 'Vendedor a domicílio, representante de vendas, vendedor de catálogo (Avon, Natura etc.)',
    9: 'Vendedor ambulante (feirante, camelô, comerciante de rua, quiosque)',
    10: 'Cozinheiro e garçom (de restaurantes, empresas)',
    11: 'Padeiro, açougueiro e doceiro',
    12: 'Agricultor, criador de animais, pescador, silvicultor e jardineiro',
    13: 'Auxiliar da agropecuária (colhedor de frutas, boia fria, etc.)',
    14: 'Motorista (de aplicativo, de taxi, de van, de mototáxi, de ônibus)',
    15: 'Motorista de caminhão (caminhoneiro)',
    16: 'Motoboy',
    17: 'Entregador de mercadorias (de restaurante, de farmácia, de loja, Uber Eats, IFood, Rappy etc.)',
    18: 'Pedreiro, servente de pedreiro, pintor, eletricista, marceneiro',
    19: 'Mecânico de veículos, máquinas industriais etc.',
    20: 'Artesão, costureiro e sapateiro',
    21: 'Cabeleireiro, manicure e afins',
    22: 'Operador de máquinas, montador na indústria',
    23: 'Auxiliar de produção, de carga e descarga',
    24: 'Professor da educação infantil, de ensino fundamental, médio ou superior',
    25: 'Pedagogo, professor de idiomas, música, arte e reforço escolar',
    26: 'Médico, enfermeiro, profissionais de saúde de nível superior',
    27: 'Técnico, profissional da saúde de nível médio',
    28: 'Cuidador de crianças, doentes ou idosos',
    29: 'Segurança, vigilante, outro trabalhador dos serviços de proteção',
    30: 'Policial civil',
    31: 'Porteiro, zelador',
    32: 'Artista, religioso (padre, pastor etc.)',
    33: 'Diretor, gerente, cargo político ou comissionado',
    34: 'Outra profissão de nível superior (advogado, engenheiro, contador, jornalista etc.)',
    35: 'Outro técnico ou profissional de nível médio',
    36: 'Outros'
}

mapeamento_atividade_empresa = {
    1: 'Agricultura, pecuária, produção florestal e pesca',
    2: 'Extração de petróleo, carvão mineral, minerais metálicos, pedra, areia, sal etc.',
    3: 'Indústria da transformação (inclusive confecção e fabricação caseira)',
    4: 'Fornecimento de eletricidade e gás, água, esgoto e coleta de lixo',
    5: 'Construção',
    6: 'Comércio no atacado e varejo',
    7: 'Reparação de veículos automotores e motocicletas',
    8: 'Transporte de passageiros',
    9: 'Transporte de mercadorias',
    10: 'Armazenamento, correios e serviços de entregas',
    11: 'Hospedagem (hotéis, pousadas etc.)',
    12: 'Serviço de alimentação (bares, restaurantes, ambulantes de alimentação)',
    13: 'Informação e comunicação (jornais, rádio e televisão, telecomunicações e informática)',
    14: 'Bancos, atividades financeiras e de seguros',
    15: 'Atividades imobiliárias',
    16: 'Escritórios de advocacia, engenharia, publicidade e veterinária (Atividades profissionais, científicas e técnicas)',
    17: 'Atividades de locação de mão de obra, segurança, limpeza, paisagismo e teleatendimento',
    18: 'Administração pública (governo federal, estadual e municipal)',
    19: 'Educação',
    20: 'Saúde humana e assistência social',
    21: 'Organizações religiosas, sindicatos e associações',
    22: 'Atividade artísticas, esportivas e de recreação',
    23: 'Cabeleireiros, tratamento de beleza e serviços pessoais',
    24: 'Serviço doméstico remunerado (será imputado da posição na ocupação)',
    25: 'Outro'
}

mapeamento_faixa_rendimento = {
    0: '0 - 100',
    1: '101 - 300',
    2: '301 - 600',
    3: '601 - 800',
    4: '801 - 1.600',
    5: '1.601 - 3.000',
    6: '3.001 - 10.000',
    7: '10.001 - 50.000',
    8: '50.001 - 100.000',
    9: 'Mais de 100.000'
}

mapeamento_tipo_domicilio = {
    1: 'Próprio - já pago',
    2: 'Próprio - ainda pagando',
    3: 'Alugado',
    4: 'Cedido por empregador',
    5: 'Cedido por familiar',
    6: 'Cedido de outra forma',
    7: 'Outra condição'
}

In [ ]:
def traducoes(df):
    # Aplicar as traduções para cada coluna
    df['Condição no domicílio'] = df['Condição no domicílio'].map(mapeamento_condicao_domicilio)
    df['Cor ou raça'] = df['Cor ou raça'].map(mapeamento_cor_raca)
    df['Escolaridade'] = df['Escolaridade'].map(mapeamento_escolaridade)
    df['Sexo'] = 
    df['Frequenta escola'] = df['Frequenta escola'].map(mapeamento_frequenta_escola)
    df['A escola / escola ou faculdade que frequenta é pública ou privada?'] = df['A escola / escola ou faculdade que frequenta é pública ou privada?'].map(mapeamento_escola_publica_privada)
    df['Você está tendo aulas presenciais?'] = df['Você está tendo aulas presenciais?'].map(mapeamento_aulas_presenciais)

    # Traduzir sintomas e testes COVID-19
    colunas_sintomas = [
        'Na semana passada teve febre?', 'Na semana passada teve tosse?', 'Na semana passada teve dor de garganta?',
        'Na semana passada teve dificuldade para respirar?', 'Na semana passada teve dor de cabeça?',
        'Na semana passada teve dor no peito?', 'Na semana passada teve náusea?', 'Na semana passada teve nariz entupido ou escorrendo?',
        'Na semana passada teve fadiga?', 'Na semana passada teve dor nos olhos?', 'Na semana passada teve perda de cheiro ou sabor?',
        'Na semana passada teve dor muscular?', 'Na semana passada teve diarreia?'
    ]
    for coluna in colunas_sintomas:
        df[coluna] = df[coluna].map(mapeamento_sintomas)

    colunas_testes = [
        'O(A) Sr(a) fez algum teste para saber se estava infectado(a) pelo coronavírus?',
        'Fez o exame coletado com cotonete na boca e/ou nariz (SWAB)?',
        'Fez o exame de coleta de sangue através de furo no dedo?',
        'Fez o exame de coleta de sangue através da veia do braço?'
    ]
    for coluna in colunas_testes:
        df[coluna] = df[coluna].map(mapeamento_teste_covid)

    #colunas_resultados = [
   #     'Qual o resultado?',  # Aplica para SWAB, sangue no dedo e sangue na veia
   # ]
    #for coluna in colunas_resultados:
       # df[coluna] = df[coluna].map(mapeamento_resultado_teste)

    # Traduzir informações de trabalho
    df['No trabalho (único ou principal) que tinha nessa semana, era:'] = df['No trabalho (único ou principal) que tinha nessa semana, era:'].map(mapeamento_tipo_trabalho)
    df['Esse trabalho era na área:'] = df['Esse trabalho era na área:'].map(mapeamento_area_trabalho)
    df['Tem carteira de trabalho assinada ou é funcionário público estatutário?'] = df['Tem carteira de trabalho assinada ou é funcionário público estatutário?'].map(mapeamento_carteira_trabalho)
    df['Que tipo de trabalho, cargo ou função você realiza no seu trabalho (único ou principal)?'] = df['Que tipo de trabalho, cargo ou função você realiza no seu trabalho (único ou principal)?'].map(mapeamento_tipo_cargo)
    df['Qual é a principal atividade do local ou empresa em que você trabalha?'] = df['Qual é a principal atividade do local ou empresa em que você trabalha?'].map(mapeamento_atividade_empresa)

    # Traduzir rendimento e domicílio
    df['Número da faixa do rendimento/retirada em dinheiro'] = df['Número da faixa do rendimento/retirada em dinheiro'].map(mapeamento_faixa_rendimento)
    df['Este domicílio é:'] = df['Este domicílio é:'].map(mapeamento_tipo_domicilio)

    return df

Tratamento dos dados

In [7]:
import pandas as pd

for i in range(11, 12):
    # ler o arquivo original
    df = pd.read_csv(f"dados-originais/pc-{i:0>2}.csv")
    
    # seleciona apenas os dados da Paraíba (código 25)
    df = df[df['UF'] == 25]

    # seleciona as colunas que existirem
    # nos meses iniciais da pesquisa os campos sobre testes de coronavírus ainda não exisitiam
    # então vamos selecionar o máximo de colunas que conseguirmos de cada mês
    colunas_existentes = {k: v for k, v in colunas.items() if k in df.columns}

    # filtra e renomeia as colunas
    df_filtrado = df[list(colunas_existentes.keys())].rename(columns=colunas_existentes)

    # Tradução dos dados
    df_traduzido = traducoes(df_filtrado)

    # Salva os dados em arquivos
    df_traduzido.to_csv(f'dados-mensais/pc-{i:0>2}.csv', index=False, encoding="latin1", sep=";")

In [8]:
display(df_traduzido)

,Estrato,Situação do domicílio,Condição no domicílio,Idade do morador,Sexo,Cor ou raça,Escolaridade,Frequenta escola,A escola / escola ou faculdade que frequenta é pública ou privada?,Você está tendo aulas presenciais?,...,Fez o exame de coleta de sangue através da veia do braço?,Qual o resultado?,"No trabalho (único ou principal) que tinha nessa semana, era:",Esse trabalho era na área:,Tem carteira de trabalho assinada ou é funcionário público estatutário?,"Que tipo de trabalho, cargo ou função você realiza no seu trabalho (único ou principal)?",Qual é a principal atividade do local ou empresa em que você trabalha?,Número da faixa do rendimento/retirada em dinheiro,Valor em dinheiro,Este domicílio é:
100336,2510011,1,Pessoa responsável pelo domicílio,43,2,Parda,Médio completo,NaN,NaN,NaN,...,NaN,NaN,Empregado do setor privado,NaN,Não,"Vendedor ambulante (feirante, camelô, comercia...","Serviço de alimentação (bares, restaurantes, a...",0 - 100,40.0,Cedido por familiar
100337,2510011,1,Filho(a) somente do responsável,18,2,Parda,Médio incompleto,Sim,Pública,"Não, meu curso é online",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cedido por familiar
100338,2510011,1,Pessoa responsável pelo domicílio,58,1,Preta,Fundamental incompleto,NaN,NaN,NaN,...,NaN,NaN,Conta própria,NaN,NaN,"Vendedor a domicílio, representante de vendas,...",Comércio no atacado e varejo,801 - 1.600,1500.0,Próprio - já pago
100339,2510011,1,Cônjuge ou companheiro(a) de sexo diferente,47,2,Parda,Fundamental incompleto,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Próprio - já pago
100340,2510011,1,Sogro(a),83,2,Parda,Sem instrução,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Próprio - já pago
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110204,2553022,2,Pessoa responsável pelo domicílio,74,2,Branca,Fundamental incompleto,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cedido de outra forma
110205,2553022,2,Pessoa responsável pelo domicílio,50,1,Parda,Sem instrução,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cedido de outra forma
110206,2553022,2,Pessoa responsável pelo domicílio,35,2,Parda,Médio completo,NaN,NaN,NaN,...,NaN,NaN,"Trabalhador doméstico (empregado doméstico, cu...",NaN,Não,"Empregado doméstico, diarista, cozinheiro (em ...",Serviço doméstico remunerado (será imputado da...,301 - 600,500.0,Cedido por familiar
110207,2553022,2,Cônjuge ou companheiro(a) de sexo diferente,36,1,Parda,Fundamental incompleto,NaN,NaN,NaN,...,NaN,NaN,Empregado do setor público (inclusive empresas...,Municipal,Não,"Mecânico de veículos, máquinas industriais etc.",Reparação de veículos automotores e motocicletas,1.601 - 3.000,2000.0,Cedido por familiar


##### Tratamento dos dados

In [ ]:
import pandas as pd
from os import mkdir

dados_mensais = list()
for mes in range(5, 12):
    # lendo todos os dados do mês
    df = pd.read_csv(f"dados-mensais/pc-{mes:0>2}.csv")
    

    # filtrando para a paraíba (código 25)
    df_paraiba = df[df['UF'] == 25]

    # filtrando as colunas que analisaremos
    display(df_paraiba)

In [ ]:
# ANÁLISE DE RENDA
colunas_desejadas = ["UF","B0011", "B0012", "B0013", "B0014", "B0015", "B0016",
                      "B0017", "B0018", "B0019", "B00110", "B00111", "B00112",
                      "C01011", "C01012"]
df_renda = df_paraiba[colunas_desejadas]


display(df_renda)

try: 
    mkdir('resultados')
except FileExistsError:
    print('Diretório \"resultados\" já existe.')

df_renda.to_csv("resultados/pb-05-contaminacaoXrenda.csv", index=False, encoding="latin1", sep=";")

In [ ]:
# ANÁLISE DE INSTRUÇÃO
colunas_desejadas = ["UF","B0011", "B0012", "B0013", "B0014", "B0015", "B0016",
                      "B0017", "B0018", "B0019", "B00110", "B00111", "B00112",
                      "A005"]

df_renda = df_paraiba[colunas_desejadas]

display(df_renda)

try: 
    mkdir('resultados')
except FileExistsError:
    print('Diretório \"resultados\" já existe.')

df_renda.to_csv("resultados/pb-05-contaminacaoXescolaridade.csv", index=False, encoding="latin1", sep=";")